# Obteniendo el número de muertes confirmadas por COVID-19 por semana epidemiológica de cada uno de los 24 departamentos del Perú 

## 0. Cargar librerías

In [3]:
import pandas as pd
import numpy as np

import functions as fn

## 1. Cargar direcciones de RawData

In [1]:
vac_url = 'RawData/TB_VACUNACION_COVID19.csv'
fal_url = 'RawData/fallecidos_covid.csv'

## 2. Procesar del dataset de fallecidos

### 2.1. Leer el dataset

In [4]:
# Leer solamente la columna fecha de fallecimiento y departamento
fal_col = ['FECHA_FALLECIMIENTO', 'DEPARTAMENTO']
df_fal =  pd.read_csv(fal_url, sep = ';', usecols = fal_col, dtype = {'FECHA_FALLECIMIENTO':'int32', 'DEPARTAMENTO': 'category'})
del fal_col

# Transformar a formato fecha
df_fal.loc[:,'FECHA_FALLECIMIENTO'] = pd.to_datetime(df_fal['FECHA_FALLECIMIENTO'], format = '%Y%m%d')
df_fal.head()

,FECHA_FALLECIMIENTO,DEPARTAMENTO
0,2021-06-11,LAMBAYEQUE
1,2021-03-17,PIURA
2,2021-06-02,ICA
3,2021-07-03,AREQUIPA
4,2021-05-06,LIMA


### 2.2. Procesar el dataset

Se añaden 3 columnas. El año, la semana epidemiológica correspondiente y un contador para contar cada caso de fallecimiento por COVID-19. Se elimina la columna **'fecha_fallecimiento'**.

In [5]:
# Agregamos el año y semana epidemilógica de cada fallecido
fn.date_to_epiweek(df_fal,'FECHA_FALLECIMIENTO')
                 
df_fal = df_fal.astype({'year': 'int16', 'epi_week': 'int8'})

del df_fal['FECHA_FALLECIMIENTO']
df_fal.head()

,DEPARTAMENTO,year,epi_week
0,LAMBAYEQUE,2021,23
1,PIURA,2021,11
2,ICA,2021,22
3,AREQUIPA,2021,26
4,LIMA,2021,18


**IMPORTANTE:** No tenemos el total de fallecidos por departamento y semana epidemiológica, sino el departamento y semana epidemiológica de cada caso.

### 2.3. Realizar una crosstab de fallecidos por COVID-19 por departamento de Perú, para cada año y semana epidemiológica

Esto se lo realiza para obtener el TOTAL de fallecidos por cada departamento y semana epidemiológica

In [ ]:
falxdep = pd.crosstab(index=[df_fal['year'], df_fal['epi_week']],
                      columns=[df_fal['DEPARTAMENTO']],
                      margins = False)

falxdep.head()

DEPARTAMENTO   AMAZONAS  ANCASH  APURIMAC  ...  TACNA  TUMBES  UCAYALI
year epi_week                              ...                        
2020 10               0       0         0  ...      0       0        0
     11               0       0         0  ...      0       0        0
     12               0       0         0  ...      0       0        2
     13               0       4         0  ...      0       0        3
     14               0       0         0  ...      1       3       11

[5 rows x 25 columns]

Ahora tenemos el TOTAL de fallecidos por cada departamento y semana epidemiológica

# Obteniendo el número de personas completamente vacunadas (2 dosis) contra COVID-19 por semana epidemiológica de cada uno de los 24 departamentos del Perú 

## 0. Cargar librerías

In [14]:
import pandas as pd
import numpy as np

import functions as fn

## 1. Cargar el dataset (vacxdep) de personas COMPLETAMENTE VACUNADAS (2 dosis) por los 24 departamentos del Perú

In [15]:
def vac_department(vac_url):
    """
    Función que toma la dirección del dataset de vacunados y devuelve el número de VACUNADOS 
    por los 24 departamentos del Perú
    """
    vac_col = ['id_centro_vacunacion', 'dosis','fecha_vacunacion']                     
    df_vac = fn.read_largeCSV_file(vac_url, ',', vac_col)    
    lst_vac = fn.df_into_chunks(df_vac)               
    
    for df in lst_vac:                                           
        df = df.drop(df[df["dosis"] == 1].index,  inplace=True)     # Drop non fully vaccinated (1 dose)

    for df in lst_vac:
        df['vacunado'] = 1  # To count each case
        df['vacunado'] = df['vacunado'].apply(np.int8)
        del df['dosis']     # Dose var is no needed anymore

    return lst_vac

In [16]:
vac_url = "RawData/TB_VACUNACION_COVID19.csv"
vacxdep = vac_department(vac_url)
del vac_url

/home/xxotto/anaconda3/lib/python3.8/site-packages/pandas/core/frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(
/home/xxotto/anaconda3/lib/python3.8/site-packages/pandas/core/frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(
/home/xxotto/anaconda3/lib/python3.8/site-packages/pandas/core/frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(
/hom

## 0. Usamos la data incial de vacunados por departamento (vacxdep)

In [17]:
print("Head of the first chunk of all fully vaccinated people in Peru (2 doses) per place of vaccination:")
print(vacxdep[0].head())

Head of the first chunk of all fully vaccinated people in Peru (2 doses) per place of vaccination:
  fecha_vacunacion  id_centro_vacunacion  vacunado
0       19/07/2021                    17         1
1       17/06/2021                  1828         1
2       11/06/2021                103617         1
3       28/07/2021                   891         1
7        5/06/2021                108281         1


## 1. Transformamos a formato fecha (yyyy-mm-dd) la columna "fecha_vacunacion" de todos los chunks

In [18]:
for i, chunk in enumerate(vacxdep):
    chunk = fn.variable_fecha_ymd(chunk, "fecha_vacunacion")
    print(vacxdep[i].head())

  fecha_vacunacion  id_centro_vacunacion  vacunado
0       2021-07-19                    17         1
1       2021-06-17                  1828         1
2       2021-06-11                103617         1
3       2021-07-28                   891         1
7       2021-06-05                108281         1
       fecha_vacunacion  id_centro_vacunacion  vacunado
500001       2021-10-03                   660         1
500002       2021-09-05                   674         1
500009       2021-10-02                  2679         1
500010       2021-09-08                  3781         1
500012       2021-06-23                105201         1
        fecha_vacunacion  id_centro_vacunacion  vacunado
1000002       2021-08-29                   248         1
1000007       2021-07-28                  2034         1
1000009       2021-03-31                115024         1
1000010       2021-10-09                   186         1
1000011       2021-09-11                   121         1
       fecha_vac

## 2. Obtenemos la semana epidemiológica y año de la columna "fecha_vacunacion" ahora en formato fecha (yyyy-mm-dd) de todos los chunks

In [19]:
for i, chunk in enumerate(vacxdep):
    chunk = fn.date_to_epiweek(chunk, "fecha_vacunacion")

print(vacxdep[0].head())

   id_centro_vacunacion  vacunado  epi_year  epi_week
0                    17         1      2021        29
1                  1828         1      2021        24
2                103617         1      2021        23
3                   891         1      2021        30
7                108281         1      2021        22


## 3. Reemplazar id_centro_vacunacion con el departamento respectivo

In [20]:
ubigeo_url = 'RawData/TB_UBIGEOS.csv'
vaccenter_url = 'RawData/TB_CENTRO_VACUNACION.csv'

ubigeo = pd.read_csv(ubigeo_url, usecols = ['id_ubigeo', 'departamento'])
vaccenter = pd.read_csv(vaccenter_url, usecols= ['id_centro_vacunacion','id_ubigeo'])

del ubigeo_url, vaccenter_url

vaccenter = vaccenter.merge(ubigeo, on = 'id_ubigeo', how = 'left')
del vaccenter['id_ubigeo']

print("Head of the merged dataframe (vaccenter) with: 'id_centro_vacunacion' and 'departamento'") 
print(vaccenter.head(10))

Head of the merged dataframe (vaccenter) with: 'id_centro_vacunacion' and 'departamento'
   id_centro_vacunacion departamento
0                  2021         PUNO
1                  3699   SAN MARTIN
2                   154   SAN MARTIN
3                   155   SAN MARTIN
4                  3260   SAN MARTIN
5                  2906       ANCASH
6                  2907       ANCASH
7                  2909       ANCASH
8                  2910       ANCASH
9                  2912       ANCASH


## 4.

In [21]:

df = vacxdep[0].merge(vaccenter, on = 'id_centro_vacunacion', how = 'left')
#del vacxdep_id['id_centro_vacunacion']

print(df.head())

   id_centro_vacunacion  vacunado  epi_year  epi_week departamento
0                    17         1      2021        29         LIMA
1                  1828         1      2021        24   LAMBAYEQUE
2                103617         1      2021        23     AYACUCHO
3                   891         1      2021        30         LIMA
4                108281         1      2021        22   LAMBAYEQUE


In [22]:
for i, chunk in enumerate(vacxdep):
    chunk = chunk.merge(vaccenter, on = 'id_centro_vacunacion', how = 'left')
    vacxdep[i] = chunk

print(vacxdep[0].head())

   id_centro_vacunacion  vacunado  epi_year  epi_week departamento
0                    17         1      2021        29         LIMA
1                  1828         1      2021        24   LAMBAYEQUE
2                103617         1      2021        23     AYACUCHO
3                   891         1      2021        30         LIMA
4                108281         1      2021        22   LAMBAYEQUE


## 3. Unimos nu

In [23]:
def vacxdep_chunks(dfs_vac):
    """
    Devuelve un dataframe con el TOTAL DE VACUNADOS por DEPARTAMENTO 
    del PERÚ (recibe una lista de dataframes o chunks)
    """
    var_holder = {}     # Diccionario para guardar nombres                                             
    lst_epi_vac = []    # Lista de dfs para cada sumatoria de chunks
                                         
    for i, chunk in enumerate(dfs_vac):
        var_holder['epi_vac_' + str(i)]= pd.crosstab(index = [chunk['epi_year'], chunk['epi_week']],
                                                     columns = [chunk['departamento']])
        lst_epi_vac.append(var_holder['epi_vac_' + str(i)])

    merged_epivac = pd.concat(lst_epi_vac, axis=1)  # Merge all dfs
    merged_epivac = merged_epivac.fillna(0).astype(np.int64)
    merged_epivac = merged_epivac.groupby(level=0, axis=1).sum()
    #epi_vac = pd.DataFrame(merged_epivac.sum(numeric_only=True, axis=1))
    #epi_vac.columns = ['vacunados']
    #epi_vac['vacunados'] = epi_vac['vacunados'].astype(np.int64)
    #epi_vac.reset_index(level=0, inplace=True)

    return merged_epivac

In [24]:
vacxdepxsemEpi = vacxdep_chunks(vacxdep)
print(vacxdepxsemEpi.head(50))

departamento       AMAZONAS  ANCASH  APURIMAC  AREQUIPA  AYACUCHO  CAJAMARCA  \
epi_year epi_week                                                              
2021     6                0       0         0         0         0          0   
         7                0       0         0         0         0          0   
         8                0       2         0         0         0          0   
         9              887    1974      2494      3763      2362       2726   
         10             528    1083       833      4584      2804        939   
         11            2724    5543      2809      6466      1421       5081   
         12             351    1550       489      2037       776       2251   
         13             170     324       371       336       242        402   
         14             174     543       328      2859       338        534   
         15              95    4555       130     11599       124        226   
         16             120     439     

Guardar el dataframe:

In [25]:
vacxdepxsemEpi.to_csv('Data/vacunados_x_departamentos_x_semanaEpi.csv')